# Earnings Call Project: Data Cleaning
<br>
CIS 831 Deep Learning – Term Project<br>
Kansas State University
<br><br>
James Chapman<br>
John Woods<br>
Nathan Diehl<br>
<br>

This notebook uses the transformer architecture from the [HTML paper](https://www.researchgate.net/publication/340385140_HTML_Hierarchical_Transformer-based_Multi-task_Learning_for_Volatility_Prediction). 

From the previous notebooks, we have features for both audio and text, of each sentence, of each meeting.<br>
- audio 
    - PRAAT (27 features)
- text
    - GLOVE (300 features)
    - ROBERTA (1024 features)
    - ROBERTA with averaging (1024 features)
    - FinLang investopedia (768 features) from huggingface sentencetransformers
    - BGE financial (1024 features) from huggingface sentencetransformers

This notebook performs 3 nested loops. Each audio/text pair, and for each N_DAYS, we train models with 17 different alpha values. The alpha value the lowest MSE, based on the validation set, is been used to train a model with Both training and validation sets. This model is finally tested on the test set.

These notebooks also give us a playground to work with the data, and test the models. Now we can insert sentiment detection, segmentation of the text, etc.

In [90]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn.functional as F
from torch import nn
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

np.random.seed(777)
torch.manual_seed(777)
torch.cuda.manual_seed_all(777)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [91]:
import shutil
import os

#fresh start

directory_path = r"C:\Users\James\OneDrive\Documents\GitHub\Earnings_call_project\runs"
if os.path.exists(directory_path):
    shutil.rmtree(directory_path)
    
path = "C:/Users/James/AppData/Local/Temp/.tensorboard-info/"
for filename in os.listdir(path):
    filepath = os.path.join(path, filename)
    print(filepath)
    os.remove(filepath)

In [92]:
directories = ['Roberta/', 'Roberta2/', 'investopedia/', 'bge/', 'glove/']#, 'bge_base/'
all_n_days = ['3', '7', '15', '30']
alphas = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85]

# static hyper-parameters from paper
max_norm = 1.0
num_epochs = 10
warmup_steps = 1000
batch_size=4
dropout=0.5
heads=2
depth=2

lr=0.000002 # 1/10 learning rate of paper
is_using_scaler = True # whether or not to use standardscaler

# This is the hierarchical transformer from the HTML paper.

## This is taken directly from the GitHub account for HTML paper.
- a few adaptions are highlighted with ##############################################


- [GitHub HTML - util.py](https://github.com/YangLinyi/HTML-Hierarchical-Transformer-based-Multi-task-Learning-for-Volatility-Prediction/blob/master/Model/Sentence-Level-Transformer/transformers/util/util.py)
- [GitHub HTML - transformers_gpu.py](https://github.com/YangLinyi/HTML-Hierarchical-Transformer-based-Multi-task-Learning-for-Volatility-Prediction/blob/master/Model/Sentence-Level-Transformer/transformers/transformers_gpu.py)

In [93]:

# https://github.com/YangLinyi/HTML-Hierarchical-Transformer-based-Multi-task-Learning-for-Volatility-Prediction/blob/master/Model/Sentence-Level-Transformer/transformers/transformers_gpu.py
def mask_(matrices, maskval=0.0, mask_diagonal=True):
    """
    Masks out all values in the given batch of matrices where i <= j holds,
    i < j if mask_diagonal is false

    In place operation

    :param tns:
    :return:
    """

    b, h, w = matrices.size()

    indices = torch.triu_indices(h, w, offset=0 if mask_diagonal else 1)
    matrices[:, indices, indices[1]] = maskval

def contains_nan(tensor):
    return bool((tensor != tensor).sum() > 0)

# https://github.com/YangLinyi/HTML-Hierarchical-Transformer-based-Multi-task-Learning-for-Volatility-Prediction/blob/master/Model/Sentence-Level-Transformer/transformers/transformers_gpu.py
class SelfAttention(nn.Module):
    def __init__(self, emb, heads=8, mask=False):
        """
        :param emb:
        :param heads:
        :param mask:
        """

        super().__init__()

        self.emb = emb
        self.heads = heads
        self.mask = mask

        self.tokeys = nn.Linear(emb, emb * heads, bias=False)
        self.toqueries = nn.Linear(emb, emb * heads, bias=False)
        self.tovalues = nn.Linear(emb, emb * heads, bias=False)

        self.unifyheads = nn.Linear(heads * emb, emb)

    def forward(self, x):

        b, t, e = x.size()
        h = self.heads
        assert e == self.emb

        keys    = self.tokeys(x)   .view(b, t, h, e)
        queries = self.toqueries(x).view(b, t, h, e)
        values  = self.tovalues(x) .view(b, t, h, e)

        # compute scaled dot-product self-attention

        # - fold heads into the batch dimension
        keys = keys.transpose(1, 2).contiguous().view(b * h, t, e)
        queries = queries.transpose(1, 2).contiguous().view(b * h, t, e)
        values = values.transpose(1, 2).contiguous().view(b * h, t, e)

        queries = queries / (e ** (1/4))
        keys    = keys / (e ** (1/4))
        # - Instead of dividing the dot products by sqrt(e), we scale the keys and values.
        #   This should be more memory efficient

        # - get dot product of queries and keys, and scale
        dot = torch.bmm(queries, keys.transpose(1, 2))

        assert dot.size() == (b*h, t, t)

        if self.mask: # mask out the lower half of the dot matrix,including the diagonal
            mask_(dot, maskval=float('-inf'), mask_diagonal=False)

        dot = F.softmax(dot, dim=2) # dot now has row-wise self-attention probabilities

        ##############################################
        ##############################################
        # assert not util.contains_nan(dot[:, 1:, :]) # only the forst row may contain nan
        assert not contains_nan(dot[:, 1:, :]) # only the forst row may contain nan 
        ##############################################
        ##############################################
        
        if self.mask == 'first':
            dot = dot.clone()
            dot[:, :1, :] = 0.0
            # - The first row of the first attention matrix is entirely masked out, so the softmax operation results
            #   in a division by zero. We set this row to zero by hand to get rid of the NaNs

        # apply the self attention to the values
        out = torch.bmm(dot, values).view(b, h, t, e)

        # swap h, t back, unify heads
        out = out.transpose(1, 2).contiguous().view(b, t, h * e)

        return self.unifyheads(out)

class TransformerBlock(nn.Module):
    def __init__(self, emb, heads, mask, seq_length, ff_hidden_mult=4, dropout=0.0):
        super().__init__()

        self.attention = SelfAttention(emb, heads=heads, mask=mask)
        self.mask = mask

        self.norm1 = nn.LayerNorm(emb)
        self.norm2 = nn.LayerNorm(emb)

        self.ff = nn.Sequential(
            nn.Linear(emb, ff_hidden_mult * emb),
            nn.ReLU(),
            nn.Linear(ff_hidden_mult * emb, emb)
        )

        self.do = nn.Dropout(dropout)

    def forward(self, x):

        attended = self.attention(x)

        x = self.norm1(attended + x)

        x = self.do(x)

        fedforward = self.ff(x)

        x = self.norm2(fedforward + x)

        x = self.do(x)

        return x

class RTransformer(nn.Module):
    """
    Transformer for sequences Regression    
    
    """

    def __init__(self, emb, heads, depth, seq_length, num_tokens, num_classes, max_pool=True, dropout=0.0):
        """
        emb: Embedding dimension
        heads: nr. of attention heads
        depth: Number of transformer blocks
        seq_length: Expected maximum sequence length
        num_tokens: Number of tokens (usually words) in the vocabulary
        num_classes: Number of classes.
        max_pool: If true, use global max pooling in the last layer. If false, use global
                         average pooling.
        """
        super().__init__()

        self.num_tokens, self.max_pool = num_tokens, max_pool

        #self.token_embedding = nn.Embedding(embedding_dim=emb, num_embeddings=num_tokens)
        self.pos_embedding = nn.Embedding(embedding_dim=emb, num_embeddings=seq_length)

        tblocks = []
        for i in range(depth):
            tblocks.append(
                TransformerBlock(emb=emb, heads=heads, seq_length=seq_length, mask=False, dropout=dropout))

        self.tblocks = nn.Sequential(*tblocks)

        self.toprobs = nn.Linear(emb, num_classes)
        self.toprobs_b = nn.Linear(emb, num_classes)
        self.do = nn.Dropout(dropout)

    def forward(self, x):
        """
        :param x: A batch by sequence length integer tensor of token indices.
        :return: predicted log-probability vectors for each token based on the preceding tokens.
        """
        sentences_emb = x
        b, t, e = x.size()

        ##############################################
        ##############################################
        # swap d() for device
        # positions = self.pos_embedding(torch.arange(t, device=d()))[None, :, :].expand(b, t, e)
        positions = self.pos_embedding(torch.arange(t, device=device))[None, :, :].expand(b, t, e)
        ##############################################
        ##############################################
        #positions = self.pos_embedding(torch.arange(t))[None, :, :].expand(b, t, e)
        #positions = torch.tensor(positions, dtype=torch.float32)
        x = sentences_emb.cuda() + positions
        x = self.do(x)

        x = self.tblocks(x)

        x = x.max(dim=1)[0] if self.max_pool else x.mean(dim=1) # pool over the time dimension
        
        
        x_a = self.toprobs(x)
        x_b = self.toprobs_b(x)
        x_a = torch.squeeze(x_a)
        x_b = torch.squeeze(x_b)
        #print('x shape: ',x.shape)
        return x_a, x_b


# dataset builder and training loop

In [94]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, embeddings, labels, labels_b):
        self.embeddings = embeddings
        self.labels = labels
        self.labels_b = labels_b

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        emb = self.embeddings[idx]
        label = self.labels[idx]
        label_b = self.labels_b[idx]
        return emb, label, label_b 

def get_data(data_directory, n_days, no_validation_set, is_using_scaler, batch_size):
    # features
    train_features = np.load('data/{}/train_features.npy'.format(data_directory))
    val_features = np.load('data/{}/val_features.npy'.format(data_directory))
    test_features = np.load('data/{}/test_features.npy'.format(data_directory))
    # targets (n_days volatility)
    train_targets = np.load('data/{}/train_targets_{}.npy'.format(data_directory, n_days))
    val_targets = np.load('data/{}/val_targets_{}.npy'.format(data_directory, n_days))
    test_targets = np.load('data/{}/test_targets_{}.npy'.format(data_directory, n_days))
    # secondary targets (log percent change of day n)
    train_secondary_targets = np.load('data/{}/train_secondary_targets_{}.npy'.format(data_directory, n_days))
    val_secondary_targets = np.load('data/{}/val_secondary_targets_{}.npy'.format(data_directory, n_days))
    test_secondary_targets = np.load('data/{}/test_secondary_targets_{}.npy'.format(data_directory, n_days))

    # after hyperparameters are tuned on the validation set
    # add validation set to training set
    if no_validation_set:
        train_features = np.concatenate((train_features, val_features), axis=0)
        train_targets = np.concatenate((train_targets, val_targets), axis=0)
        train_secondary_targets = np.concatenate((train_secondary_targets, val_secondary_targets), axis=0)

    if is_using_scaler:
        # Scaling features
        feature_scaler = StandardScaler()
        train_features = feature_scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
        val_features = feature_scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
        test_features = feature_scaler.transform(test_features.reshape(-1, test_features.shape[-1])).reshape(test_features.shape)
        # Scaling primary targets
        target_scaler = StandardScaler()
        train_targets = target_scaler.fit_transform(train_targets.reshape(-1, 1)).reshape(train_targets.shape)
        val_targets = target_scaler.transform(val_targets.reshape(-1, 1)).reshape(val_targets.shape)
        test_targets = target_scaler.transform(test_targets.reshape(-1, 1)).reshape(test_targets.shape)
        # Scaling secondary targets
        secondary_target_scaler = StandardScaler()
        train_secondary_targets = secondary_target_scaler.fit_transform(train_secondary_targets.reshape(-1, 1)).reshape(train_secondary_targets.shape)
        val_secondary_targets = secondary_target_scaler.transform(val_secondary_targets.reshape(-1, 1)).reshape(val_secondary_targets.shape)
        test_secondary_targets = secondary_target_scaler.transform(test_secondary_targets.reshape(-1, 1)).reshape(test_secondary_targets.shape)
    # Dataset & DataLoader
    training_set = Dataset(train_features, train_targets, train_secondary_targets) 
    val_set = Dataset(val_features, val_targets, val_secondary_targets)
    test_set = Dataset(test_features, test_targets, test_secondary_targets)
    trainloader = torch.utils.data.DataLoader(training_set, batch_size=batch_size, shuffle=False, num_workers=0)
    valloader = torch.utils.data.DataLoader(val_set, batch_size=len(val_set), shuffle=False, num_workers=0)
    testloader = torch.utils.data.DataLoader(test_set, batch_size=len(test_set), shuffle=False, num_workers=0)

    print(train_features.shape, train_targets.shape, train_secondary_targets.shape) 
    print(val_features.shape, val_targets.shape, val_secondary_targets.shape) 
    print(test_features.shape, test_targets.shape, test_secondary_targets.shape) 

    # train_features.shape[2] is the number of features, need to instantiate the model
    # need target scaler to inverse transform
    return trainloader, valloader, testloader, train_features.shape[2], target_scaler 


In [95]:
# this training loop is actually reused for training, validation, and testing
def  training_loop(trainloader, loader, emb_dimensions, heads, depth,dropout, warmup_steps, 
                   batch_size, num_epochs, alphas, max_norm, lr, run_name, target_scaler):
    alphas_results = []
    for alpha in alphas:
        model = RTransformer(emb=emb_dimensions, heads=heads, depth=depth, seq_length=523, 
                            num_tokens=4000, num_classes=1, max_pool=False, dropout=dropout).to(device)
        opt = torch.optim.Adam(model.parameters(), lr=lr)
        # Linear LR warmup for the first #warmup_steps training examples
        scheduler = LambdaLR(opt, lr_lambda=lambda step: min(1.0, step / (warmup_steps/batch_size)))

        seen = 0
        min_loss = float('inf')
        val_loss_a_history =  []
        writer = SummaryWriter(log_dir= f'runs/{run_name}_{alpha}')
        progress_bar = tqdm(range(num_epochs), desc="Training Progress", unit="epoch")
        for epoch in progress_bar:
            model.train()
            epoch_seen = 0 # 1st 1000 meetings trained, increase the learning rate
            train_loss_total = 0
            for i, (inputs, labels, labels_b) in enumerate(trainloader): #training data
                seen += inputs.size(0)
                epoch_seen += inputs.size(0)
                inputs = inputs.to(device, dtype=torch.float32)
                labels = labels.to(device, dtype=torch.float32)
                labels_b = labels_b.to(device, dtype=torch.float32)
                out_a, out_b = model(inputs)

                # Compute the combined loss (multitask)
                loss_a = F.mse_loss(out_a, labels)
                loss_b = F.mse_loss(out_b, labels_b)
                loss = alpha * loss_a + (1 - alpha) * loss_b # Alpha parameter
            
                opt.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm)
                opt.step()
                if seen < warmup_steps:
                    scheduler.step()

                # we only care about the primary target
                # although we training with the scaled loss, reporting the volatility must be unscaled
                out_a_unscaled = target_scaler.inverse_transform(out_a.detach().cpu().numpy().reshape(-1, 1)).reshape(out_a.shape)
                labels_unscaled = target_scaler.inverse_transform(labels.detach().cpu().numpy().reshape(-1, 1)).reshape(labels.shape)
                loss_a_unscaled = F.mse_loss(torch.tensor(out_a_unscaled, device=out_a.device), torch.tensor(labels_unscaled, device=labels.device))
                train_loss_total += loss_a_unscaled.item() * inputs.size(0) # MSE of volatility training

            # Epoch average of unscaled training loss
            train_loss_avg = train_loss_total / epoch_seen
            
            # Epoch Evaluation
            model.eval()
            val_loss_a_total = 0.0
            val_loss_b_total = 0.0
            epoch_seen = 0
            with torch.no_grad():
                for i, (inputs, labels, labels_b) in enumerate(loader): #validation ortesting
                    epoch_seen += inputs.size(0)
                    inputs = inputs.to(device, dtype=torch.float32)
                    labels = labels.to(device, dtype=torch.float32)
                    labels_b = labels_b.to(device, dtype=torch.float32)

                    out_a, out_b = model(inputs)
                    #loss_a = F.mse_loss(out_a, labels)
                    loss_b = F.mse_loss(out_b, labels_b)
                    
                    out_a_unscaled = target_scaler.inverse_transform(out_a.detach().cpu().numpy().reshape(-1, 1)).reshape(out_a.shape)
                    labels_unscaled = target_scaler.inverse_transform(labels.detach().cpu().numpy().reshape(-1, 1)).reshape(labels.shape)
                    loss_a_unscaled = F.mse_loss(torch.tensor(out_a_unscaled, device=out_a.device), torch.tensor(labels_unscaled, device=labels.device))
                    val_loss_a_total += loss_a_unscaled.item() * inputs.size(0) # MSE of volatility

                    val_loss_b_total += loss_b.item() * inputs.size(0) 
                ##############################################
                val_loss_a_avg = val_loss_a_total / epoch_seen # MSE of volatility of Val/test!!
                val_loss_b_avg = val_loss_b_total / epoch_seen
            # Epoch finished
            writer.add_scalar('Train Loss/train', train_loss_avg, epoch)
            writer.add_scalar('Loss A/val', val_loss_a_avg, epoch)
            writer.add_scalar('Loss B/val', val_loss_b_avg, epoch)
            # Update progress bar description and postfix
            progress_bar.set_description(f"Epoch {epoch+1}/{num_epochs}")
            progress_bar.set_postfix({
                "Train Loss": f"{train_loss_avg:.4f}",
                "Test Loss A": f"{val_loss_a_avg:.4f}",
                "Test Loss B": f"{val_loss_b_avg:.4f}"
            })
            # for alpha optimization, collect each Epoch's average loss
            val_loss_a_history.append(val_loss_a_avg)
        # all Epochs finished for this alpha value
        # if testing, there will only be one Alpha
        # I made a sliding window of 2, so we pick the alpha with the lowest average 2 consecutive Epoeh losses
        # min_loss = min(
        #     (val_loss_a_history[i] + val_loss_a_history[i + 1]) / 2 # average 2 consecutive Epoeh losses
        #     for i in range(len(val_loss_a_history) - 1)
        # )
        # or just the Alpha with the lowest minimum loss
        min_loss = round(min(val_loss_a_history), 3)
        alphas_results.append((alpha, min_loss))
        print(alpha, min_loss)
        writer.close()
    best_alpha, lowest_val_loss = min(alphas_results, key=lambda x: x[1])
    return best_alpha, lowest_val_loss # if testING, this is the MSE  of the test set

In [96]:

def train_val_test(data_directory, n_days):
    run_name = data_directory + '_'  + n_days
    # using training set and validation set, determine optimum hyper-parameters (just alpha)
    no_validation_set = False
    trainloader,valloader,testloader,emb_dimensions, target_scaler = get_data(data_directory, n_days, no_validation_set, is_using_scaler, batch_size)
    best_alpha, _ = training_loop(trainloader, valloader, emb_dimensions, heads, depth,dropout, warmup_steps, 
                            batch_size, num_epochs, alphas, max_norm, lr, run_name, target_scaler)
    # using optimum alpha, retrain with all training/validation sets and test on testset
    test_run_name = run_name + '_test'
    no_validation_set = True
    trainloader,valloader,testloader,emb_dimensions, target_scaler = get_data(data_directory, n_days, no_validation_set, is_using_scaler, batch_size)
    _, MSE_testset = training_loop(trainloader, testloader, emb_dimensions, heads, depth, dropout, warmup_steps, 
                        batch_size, num_epochs, [best_alpha], max_norm, lr, test_run_name, target_scaler)

    print('----------------------------------------')
    print(f'run_name-----------{run_name}')
    print(f'best_alpha---------{best_alpha}')
    print(f'MSE_testset--------{MSE_testset}')
    print('----------------------------------------')
    return best_alpha, MSE_testset

In [97]:
final_results = []
for data_directory in directories: # different feature engineering features
    for n_days in all_n_days: 
        best_alpha, MSE_testset = train_val_test(data_directory, n_days)
        final_results.append([data_directory, n_days, best_alpha, MSE_testset])

(392, 523, 1051) (392,) (392,)
(56, 523, 1051) (56,) (56,)
(117, 523, 1051) (117,) (117,)


Epoch 10/10: 100%|██████████| 10/10 [00:25<00:00,  2.53s/epoch, Train Loss=0.7474, Test Loss A=0.5626, Test Loss B=5.1637]


0.05 0.556


Epoch 10/10: 100%|██████████| 10/10 [00:25<00:00,  2.51s/epoch, Train Loss=0.7351, Test Loss A=0.6318, Test Loss B=5.0527]


0.1 0.6


...

0.8 0.117


Epoch 10/10: 100%|██████████| 10/10 [00:07<00:00,  1.34epoch/s, Train Loss=0.1917, Test Loss A=0.1257, Test Loss B=0.2758]


0.85 0.126
(448, 523, 327) (448,) (448,)
(56, 523, 327) (56,) (56,)
(117, 523, 327) (117,) (117,)


Epoch 10/10: 100%|██████████| 10/10 [00:08<00:00,  1.16epoch/s, Train Loss=0.1843, Test Loss A=0.2264, Test Loss B=0.4581]

0.15 0.226
----------------------------------------
run_name-----------glove/_30
best_alpha---------0.15
MSE_testset--------0.226
----------------------------------------


In [98]:
final_results = pd.DataFrame(final_results, columns=['data_directory', 'n_days', 'best_alpha', 'MSE_testset'])
final_results.to_csv('data/final_results.csv', index=False)
final_results

,0,1,2
0,,0.05,0.725
1,_7,0.25,0.362
2,_15,0.05,0.280
3,_30,0.45,0.190
4,Roberta2/,0.05,0.734
5,Roberta2/_7,0.40,0.343
6,Roberta2/_15,0.75,0.276
7,Roberta2/_30,0.40,0.182
8,investopedia/,0.85,0.698
9,investopedia/_7,0.75,0.324


In [ ]:
stop

In [101]:
#?
final_results_pivoted = final_results.pivot(index='data_directory', columns='n_days', values=['best_alpha', 'MSE_testset'])
final_results_pivoted.columns = [f"{n_days}_{metric}" for metric, n_days in final_results_pivoted.columns]
final_results_pivoted.reset_index(inplace=True)
final_results_pivoted


,alpha3,alpha_15,alpha_30,alpha_7,MSE3,MSE_15,MSE_30,MSE_7
model,,,,,,,,
NaN,0.05,NaN,NaN,NaN,0.725,NaN,NaN,NaN
Roberta2,0.05,0.75,0.40,0.40,0.734,0.276,0.182,0.343
_15,0.05,NaN,NaN,NaN,0.280,NaN,NaN,NaN
_30,0.45,NaN,NaN,NaN,0.190,NaN,NaN,NaN
_7,0.25,NaN,NaN,NaN,0.362,NaN,NaN,NaN
bge,0.60,0.75,0.25,0.70,0.693,0.265,0.179,0.324
glove,0.05,0.60,0.15,0.25,0.743,0.284,0.226,0.372
investopedia,0.85,0.75,0.50,0.75,0.698,0.263,0.187,0.324


In [103]:
final_results

,run_name,alpha,MSE,model,n_days
0,,0.05,0.725,NaN,3
1,_7,0.25,0.362,_7,3
2,_15,0.05,0.280,_15,3
3,_30,0.45,0.190,_30,3
4,Roberta2/,0.05,0.734,Roberta2,3
5,Roberta2/_7,0.40,0.343,Roberta2,_7
6,Roberta2/_15,0.75,0.276,Roberta2,_15
7,Roberta2/_30,0.40,0.182,Roberta2,_30
8,investopedia/,0.85,0.698,investopedia,3
9,investopedia/_7,0.75,0.324,investopedia,_7


In [99]:
%load_ext tensorboard
%tensorboard --logdir runs 

In [100]:
:)

SyntaxError: unmatched ')' (1896645534.py, line 1)